In [13]:
# standar imports
import json
import pandas as pd
import re
import tqdm
import pprint as pp

In [14]:
# load the dataset from causalbenchmark/data/cladder-v1-questions.json
with open('../../data/cladder-v1-questions.json') as f:
    data = json.load(f)

In [24]:
# read the dataset as a pandas dataframe
df = pd.DataFrame(data)
df.head()   

,question_id,desc_id,given_info,question,answer,meta,reasoning
0,19,alarm-mediation-nie-model1-spec1-q1,For husbands that don't set the alarm and wive...,Does husband negatively affect alarm clock thr...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
1,23,alarm-mediation-marginal-model1-spec1-q1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
2,35,alarm-mediation-nde-model2-spec2-q1,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
3,51,alarm-mediation-marginal-model3-spec3-q1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...,no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",{'step0': 'Let X = husband; V2 = wife; Y = ala...
4,54,alarm-mediation-backadj-model3-spec3-q0,Method 1: We look at how husband correlates wi...,"To understand how husband affects alarm clock,...",no,"{'story_id': 'alarm', 'graph_id': 'mediation',...",None


In [37]:
df_reasoning = df['reasoning']
d = df_reasoning.head()[0]
reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
# replace the new line characters with spaces
reasoning = re.sub('\n', ', ', reasoning)
reasoning

'step0: Let X = husband; V2 = wife; Y = alarm clock. step1: X->V2,X->Y,V2->Y step2: E[Y_{X=0, V2=1} - Y_{X=0, V2=0}] step3: \\sum_{V2 = v} P(Y=1|X =0,V2 = v)*[P(V2 = v | X = 1) − P(V2 = v | X = 0)] step4: P(Y=1 | X=0, V2=0) = 0.06, P(Y=1 | X=0, V2=1) = 0.51, P(Y=1 | X=1, V2=0) = 0.47, P(Y=1 | X=1, V2=1) = 0.92, P(V2=1 | X=0) = 0.45, P(V2=1 | X=1) = 0.63 step5: 0.63 * (0.51 - 0.06)+ 0.45 * (0.92 - 0.47)= 0.08 end: 0.08 > 0'

In [38]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-reasoning.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        
        d: dict = row['reasoning']
        if d is not None:
            # concat the reasoning dict entries into a single string
            reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
            reasoning = re.sub('\n', ', ', reasoning)
            f.write(reasoning + '\n')

10560it [00:00, 49871.75it/s]


In [39]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-question_reasoning.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        d: dict = row['reasoning']
        if d is not None:
            # concat the reasoning dict entries into a single string
            reasoning = ' '.join([str(k) + ": " +  str(v) for k, v in d.items()])
            reasoning = re.sub('\n', ', ', reasoning)
            f.write(row['given_info'] + ' ' + row['question'] + ' '+ reasoning + '\n')
        else:
            f.write(row['given_info'] + ' ' + row['question'] + '\n')

10560it [00:00, 39950.54it/s]


In [5]:
# keep the given_info and question columns
df = df[['given_info', 'question']]
df.head()

,given_info,question
0,For husbands that don't set the alarm and wive...,Does husband negatively affect alarm clock thr...
1,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...
2,For husbands that don't set the alarm and wive...,If we disregard the mediation effect through w...
3,The overall probability of alarm set by husban...,Is ringing alarm less likely than silent alarm...
4,Method 1: We look at how husband correlates wi...,"To understand how husband affects alarm clock,..."


In [19]:
# write each row of the dataframe to a file 
with open('../../data/cladder-v1-questions.txt', 'w') as f:
    for index, row in tqdm.tqdm(df.iterrows()):
        f.write(row['given_info'] + ' ' + row['question'] + '\n')


10560it [00:00, 54765.81it/s]


## Mean perpelexities (very odd scores):

- question: 161.67
- question + reasoning: 161.67
- reasoning: 254.66



## MATH dataset preprocess 

In [6]:
# for documents in all subfolders of the  MATH/test directory append the "problem" entry to the list of problems

import os
import json
import numpy as np
import matplotlib.pyplot as plt

# path to the MATH directory
path = "../data/MATH/test"

# list of problems
problems = []

# iterate over all subfolders
for subdir, dirs, files in os.walk(path):
    for file in files:
        # if the file is a json file
        if file.endswith(".json"):
            # open the file
            with open(os.path.join(subdir, file)) as f:
                # load the json file
                data = json.load(f)
                # append the problem entry to the list of problems
                problems.append(data["problem"])

# shuffle problem and keep first 100 samples
np.random.shuffle(problems)
problems = problems[:100]

problems

['Find the coefficient of $x$ when $(7+8x)-3(2+6x+2x^2)+9(1+3x+4x^2+12x^3)-9(7-x^2-8x^3+13x^4)$ is simplified.',
 'The average of five weights is 13 grams. This set of five weights is then increased by another weight of 7 grams. What is the average of the six weights?',
 'If the point $(3,4)$ is reflected in the $x$-axis, what are the coordinates of its image?\n\n[asy]\ndraw((-5.5,0)--(5.5,0),linewidth(1));\ndraw((-5.5,0)--(5.5,0),EndArrow);\ndraw((0,-5.5)--(0,5.5),EndArrow);\ndraw((0,-5.5)--(0,5.5),linewidth(1));\ndraw((-5,-5)--(-5,5)--(5,5)--(5,-5)--cycle);\ndraw((-4,-5)--(-4,5));\ndraw((-3,-5)--(-3,5));\ndraw((-2,-5)--(-2,5));\ndraw((-1,-5)--(-1,5));\ndraw((1,-5)--(1,5));\ndraw((2,-5)--(2,5));\ndraw((3,-5)--(3,5));\ndraw((4,-5)--(4,5));\ndraw((-5,-4)--(5,-4));\ndraw((-5,-3)--(5,-3));\ndraw((-5,-2)--(5,-2));\ndraw((-5,-1)--(5,-1));\ndraw((-5,1)--(5,1));\ndraw((-5,2)--(5,2));\ndraw((-5,3)--(5,3));\ndraw((-5,4)--(5,4));\ndot((3,4));\nlabel("$x$",(5.5,0),E);\nlabel("$y$",(0,5.5),N);\nla

In [7]:
# print each element of problems as a new line in math_problems.txt while escaping \n
with open('../data/math_problems_short.txt', 'w') as f:
    for problem in problems:
        f.write(problem.replace('\n', '\\n') + '\n')